In [ ]:
from pathlib import Path

import numpy as np
import open3d as o3d
import torch

In [ ]:
import concurrent.futures
num_workers = 8  # The number of threads or processes
# mt_executor = concurrent.futures.ThreadPoolExecutor(num_workers)  # Use ThreadPoolExecutor for CPU-bound tasks.
mp_executor = concurrent.futures.ProcessPoolExecutor(num_workers)  # Use ProcessPoolExecutor for I/O-bound tasks.

In [ ]:
nia_path = Path('/datasets/nia/')
collections_path = nia_path / 'collections'
annotations_path = nia_path / 'annotations'

In [ ]:
lidar_paths = sorted(collections_path.rglob('[0-9]' * 6 + '/**/lidar/*.pcd'))
# label_paths = sorted(annotations_path.rglob('[0-9]' * 6 + '/**/lidar/*.json'))
label_paths = [
    annotations_path / p.relative_to(collections_path).with_suffix('.json')
    for p in lidar_paths
]

In [ ]:
def get_reflectivity_from_point_cloud(path):
    cloud = o3d.t.io.read_point_cloud(str(path))
    reflectivity = cloud.point['reflectivity'].numpy()
    return reflectivity

In [ ]:
with concurrent.futures.ProcessPoolExecutor(num_workers) as mp_executor:
    reflectivities = mp_executor.map(get_reflectivity_from_point_cloud, lidar_paths)
    reflectivities = list(reflectivities)
    reflectivities = np.array(reflectivities)

In [ ]:
all([
    np.allclose(reflectivities[i], reflectivities[i].astype(int))
    for i in range(len(reflectivities))
])

In [ ]:
(np.min(reflectivities), np.max(reflectivities))

### 결론: NIA 수집 데이터의 reflective 열은 0-255의 정수 값을 가진다. normalize 해줄 필요가 있다.

In [ ]:
def get_positions_from_point_cloud(path):
    cloud = o3d.t.io.read_point_cloud(str(path))
    positions = cloud.point['positions'].numpy()
    return positions

In [ ]:
with concurrent.futures.ProcessPoolExecutor(num_workers) as mp_executor:
    positions = mp_executor.map(get_positions_from_point_cloud, lidar_paths)
    positions = list(positions)
    positions = np.array(positions)

In [ ]:
import json

In [ ]:
print(lidar_paths[0].name)
print(label_paths[0].name)

In [ ]:
from scipy.spatial import cKDTree
def find_common_points(cloud1, cloud2):
    # Create a cKDTree object for each point cloud
    tree1 = cKDTree(cloud1)
    tree2 = cKDTree(cloud2)
    
    # Fast search indices of common points of two point clouds
    # ex) indices: [[tree2_index_i], [tree2_index_j], [], ...]
    # len(indices) == len(tree1)
    common_point_indices = tree1.query_ball_tree(tree2, r=0)
    
    # overlapping_indices = []
    # for i, indices in enumerate(common_point_indices):
    #     for j in indices:
    #         overlapping_indices.append((i, j))
    
    return common_point_indices

In [ ]:
def read_json(json_path):
    with open(json_path) as f:
        label = json.load(f)
    return label

def create_segment_array(label_path, cloud_points):
    label = read_json(label_path)

    segment = np.ones((cloud_points.shape[0],), dtype=int) * -1

    annotations = label['annotations']
    for annotation in annotations:
        instance_points = annotation['3D_points']
        instance_class_id = annotation['class_id']

        indices = find_common_points(instance_points, cloud_points)
        indices = np.array(indices, dtype=int).reshape(-1)
        segment[indices] = instance_class_id
    
    return segment

In [ ]:
cloud = o3d.t.io.read_point_cloud(str(lidar_paths[0]))
coord = cloud.point['positions'].numpy()
strength = cloud.point['reflectivity'].numpy() / 255

label = read_json(str(label_paths[0]))
annotations = label['annotations']

In [ ]:
all_instance_points = [annotation['3D_points'] for annotation in annotations]
all_instance_class_ids = [annotation['class_id'] for annotation in annotations]

In [ ]:
def extract_common_points(annotation):
    instance_points = annotation['3D_points']
    indices = find_common_points(instance_points, coord)
    indices = np.array(indices, dtype=int).reshape(-1)
    return indices

segment = np.ones((coord.shape[0],), dtype=int) * -1
with concurrent.futures.ProcessPoolExecutor(num_workers) as mp_executor:
    all_instance_indices = mp_executor.map(extract_common_points, annotations)
    all_instance_indices = list(all_instance_indices)

for indices, class_id in zip(all_instance_indices, all_instance_class_ids):
    segment[indices] = class_id

In [ ]:
segment = create_segment_array(str(label_paths[0]), coord)

In [ ]:
np.unique(segment)

In [ ]:
annotations = label['annotations']
for annotation in annotations:
    instance_points = annotation['3D_points']
    instance_class_id = annotation['class_id']

    indices = find_common_points(instance_points, positions)
    indices = np.array(indices, dtype=int).reshape(-1)
    print(indices)
    break

In [ ]:
[len(label['annotations'][i]['3D_points']) for i in range(len(label['annotations']))]